# Preprocessing NAM and uWRF with Predictor Variable

The functions in this notebook are used to preprocess both datasets to be ready to be inputted into the deep learning model. The first half of this notebook focuses on processing our high resolution uWRF data and the rest preprocesses the low resolution NAM data.

Cell 0: Importing necessary libraries.

Cell 1: uWRF_filter_vars_with_pred
Renames important variable to standard name, and only keeps the downscaling variable of interest and the associated predictor variable. These variables are chosen by the user. 

Cell 2: uWRF_match_dims_with_pred
Assigns new dimensions to the dataset: time, latitude, longitude. Perserves the global and variable attributes

Cell 3: uWRF_combine_seq
Combines each dataset in sequtial order along the time dimension to form the training set.

Cell 4: uWRF_val_test
Combines each 3-hourly dataset in sequential order along the time dimension to form either the validation set or test set. The user must specify which dataset they want to create with this function. 

In [1]:
import os
import glob
import xarray as xr
import numpy as np
from scipy.interpolate import griddata

In [2]:
ds = xr.open_dataset('/home/gvaillant1/aligned-data/aligned_nam_test_data.nc')
ds

<xarray.Dataset>
Dimensions:    (time: 450, latitude: 30, longitude: 30)
Coordinates:
  * time       (time) datetime64[ns] 2019-03-17T18:00:00 ... 2019-04-03T12:00:00
  * latitude   (latitude) float32 39.84 39.94 40.05 40.16 ... 42.73 42.84 42.95
  * longitude  (longitude) float32 -76.7 -76.57 -76.43 ... -72.98 -72.85 -72.71
Data variables:
    T2         (time, latitude, longitude) float32 ...
    PSFC       (time, latitude, longitude) float32 ...
Attributes:
    Conventions:          CF-1.0
    History:              created by wgrib2
    GRIB2_grid_template:  30

In [36]:
def uWRF_filter_vars_with_pred(input_dir, output_dir, variables):

    input_files = glob.glob(os.path.join(input_dir, '*'))
    
    for file in input_files:
        
        print(f"Processing file: {file}")
        ds = xr.open_dataset(file)
        
        #Keep variable choosen by the user. This should be the variable being downscaled and an associated predictor variable
        ds_filtered = ds[variables]

        #Rename variables
        ds_filtered = ds_filtered.rename({'XLAT': 'latitude', 'XLONG': 'longitude', 'XTIME': 'time'})

        filename = os.path.basename(file)
        output_file = os.path.join(output_dir, filename)
        print(f"Saving file to: {output_file}")
        ds_filtered.to_netcdf(output_file)
       
def main():
    
    for i in range(1, 32):
        remote_input_dir = f"/D4/data/gvaillant/uwrf/03/{str(i).zfill(2)}/d02_files" #Use either d02 files or d03 files
        print(f"Processing directory: {remote_input_dir}")
        
        remote_output_dir = f"/D4/data/gvaillant/prep-uwrf/d02/pred-stage1/03/{str(i).zfill(2)}"
        print(f"Output directory: {remote_output_dir}")
        
        uWRF_filter_vars_with_pred(remote_input_dir, remote_output_dir, ['T2', 'PSFC']) #User chooses the variables 
            
    print("Done processing stage1 uWRF files with predictor!")

#Uncomment below to run:
#main()

In [37]:
def uWRF_match_dims_with_pred(input_dir, output_dir):
    
    input_files = glob.glob(os.path.join(input_dir, '*'))

    for file_name in input_files:
        
        print(f"Processing file: {file_name}")
        ds = xr.open_dataset(file_name)

        #Extract latitude, longitude, and time values (these will be the dimensions of the dataset)
        lat_values = ds['latitude'].values[0, :, :]  # Use the first timestep for latitudes *CHECKKKKKKK*
        lon_values = ds['longitude'].values[0, :, :]  # Use the first timestep for longitudes
        time = ds['time']

        #Preserve attributes
        lat_attrs = ds['latitude'].attrs
        lon_attrs = ds['longitude'].attrs
        time_attrs = ds['time'].attrs

        #Reorganize dataset dimensions
        new_vars = {}
        for var_name in ds.data_vars:
            var = ds[var_name]
            new_vars[var_name] = (['time', 'latitude', 'longitude'], var.values)  #Reassign dimensions

        #Create a new dataset with updated dimensions
        new_ds = xr.Dataset(
            new_vars,
            coords={
                'latitude': (['latitude'], lat_values[:, 0]),  # Convert to 1D
                'longitude': (['longitude'], lon_values[0, :]),  # Convert to 1D
                'time': time.values
            }
        )

        #Add attributes
        new_ds['latitude'].attrs.update(lat_attrs)
        new_ds['longitude'].attrs.update(lon_attrs)
        new_ds['time'].attrs.update(time_attrs)

        #Add variable attributes
        for var_name in ds.data_vars:
            new_ds[var_name].attrs.update(ds[var_name].attrs)

        #Add global attributes
        new_ds.attrs.update(ds.attrs)

        output_file_name = os.path.basename(file_name)
        output_file_path = os.path.join(output_dir, output_file_name)
        new_ds.to_netcdf(output_file_path)
        print(f"Saved file to: {output_file_path}")

    print("Done with uWRF dimension adjustment!")


def main():
    for i in range(1, 31):
        input_dir = f"/D4/data/gvaillant/prep-uwrf/d02/pred-stage1/03/{str(i).zfill(2)}"
        print(f"Processing directory: {input_dir}")

        output_dir = f"/D4/data/gvaillant/prep-uwrf/d02/pred-stage2/03/{str(i).zfill(2)}"
        print(f"Output directory: {output_dir}")

        uWRF_match_dims_with_pred(input_dir, output_dir)
        print(f"Done processing directory: {input_dir}")

    print("Done processing stage2 uWRF files!")

#Uncomment below to run:
#main()

In [ ]:
#Use this

def adjust_bounds_to_match(lat_values, lon_values, min_lat, max_lat, min_lon, max_lon):
    """
    Adjust bounds until the number of latitude and longitude values is the same,
    neither is a prime number, and both are divisible by 4.
    """
    while True:
        lat_count = len(lat_values[(lat_values >= min_lat) & (lat_values <= max_lat)])
        lon_count = len(lon_values[(lon_values >= min_lon) & (lon_values <= max_lon)])

        # Check if BOTH conditions are met
        if lat_count == lon_count and lat_count % 4 == 0:
            break  # Conditions satisfied: same count, not prime, divisible by 4

        # Adjust bounds to change the count
        if lat_count != lon_count:
            if lat_count < lon_count:
                max_lat += (lat_values[1] - lat_values[0])  # Increment latitude bound
            else:
                max_lon += (lon_values[1] - lon_values[0])  # Increment longitude bound
        else:  # If counts are equal but not meeting the conditions
            max_lat += (lat_values[1] - lat_values[0])  # Increment latitude bound
            max_lon += (lon_values[1] - lon_values[0])  # Increment longitude bound

    return min_lat, max_lat, min_lon, max_lon


def uWRF_spatial_cut(input_dir, output_dir, min_lat, max_lat, min_lon, max_lon):
    """
    Function to spatially filter uWRF data to cover NYC, the boroughs, and ensure 
    consistent latitude and longitude dimensions, avoiding prime numbers and ensuring divisibility by 4.
    """
    input_files = glob.glob(os.path.join(input_dir, '*'))

    for file in input_files:
        print(f"Processing file: {file}")
        ds = xr.open_dataset(file)

        lat = ds['latitude']
        lon = ds['longitude']

        # Handle 1D or 2D coordinates dynamically
        if lat.ndim == 1 and lon.ndim == 1:
            # 1D Coordinates
            lat_values = lat.values
            lon_values = lon.values

            # Adjust bounds to match dimensions and avoid prime numbers
            min_lat, max_lat, min_lon, max_lon = adjust_bounds_to_match(
                lat_values, lon_values, min_lat, max_lat, min_lon, max_lon
            )

            filtered_data = ds.sel(
                latitude=slice(min_lat, max_lat),
                longitude=slice(min_lon, max_lon)
            )
        else:
            # 2D Coordinates
            lat_mask = (lat >= min_lat) & (lat <= max_lat)
            lon_mask = (lon >= min_lon) & (lon <= max_lon)
            combined_mask = lat_mask & lon_mask

            filtered_data = ds.where(combined_mask, drop=True)

            # Ensure lat/lon counts are equal, not prime, and divisible by 4
            lat_values = filtered_data['latitude'].values
            lon_values = filtered_data['longitude'].values
            min_lat, max_lat, min_lon, max_lon = adjust_bounds_to_match(
                lat_values, lon_values, min_lat, max_lat, min_lon, max_lon
            )

        # Save the output
        os.makedirs(output_dir, exist_ok=True)
        filename = os.path.basename(file)
        output_file = os.path.join(output_dir, filename)
        print(f"Saving file to: {output_file}")
        filtered_data.to_netcdf(output_file)


def main():
    for i in range(1, 29):
        input_dir = f"/D4/data/gvaillant/prep-uwrf/d02/pred-stage2/03/{str(i).zfill(2)}"
        print(f"Processing directory: {input_dir}")

        output_dir = f"/D4/data/gvaillant/prep-uwrf/d02/pred-stage3/03/{str(i).zfill(2)}"
        print(f"Output directory: {output_dir}")

        # Bounds to cover NYC and Boroughs
        min_lat = 40.4774
        max_lat = 40.9176
        min_lon = -74.2591
        max_lon = -73.7004

        uWRF_spatial_cut(input_dir, output_dir, min_lat, max_lat, min_lon, max_lon)
        print(f"Done spatially filtering the uWRF files to NYC!")


#main()

In [13]:
import os
import glob
import xarray as xr

def uWRF_combine_seq(input_dirs, output_dir):
    combined_datasets = []
    
    for input_dir in input_dirs:
        # Loop through each day's subdirectory within the monthly directory
        day_dirs = sorted(next(os.walk(input_dir))[1])  # Get list of day subdirectories
        for day_dir in day_dirs:
            day_path = os.path.join(input_dir, day_dir)  # Full path to each day's subdirectory
            input_files = sorted(glob.glob(os.path.join(day_path, '*')))  # Gather all files in the daily subdirectory
            
            # Load each file into a dataset and add to the list
            datasets = [xr.open_dataset(file) for file in input_files]
            combined_datasets.extend(datasets)

    # Concatenate all datasets along the 'time' dimension
    combined_dataset = xr.concat(combined_datasets, dim='time')

    # Generate output filename based on month range from input directories
    month_range = "-".join([os.path.basename(month_dir) for month_dir in input_dirs])
    output_file_name = f'uWRF_final_{month_range}.nc'
    output_file_path = os.path.join(output_dir, output_file_name)

    # Save the concatenated dataset to a NetCDF file
    combined_dataset.to_netcdf(output_file_path, encoding={'time': {'units': 'hours since 2019-10-11'}}) #that isnt correct
    print(f'Combined dataset saved to {output_file_path}')

def main():
    input_dirs = [
        "/D4/data/gvaillant/prep-uwrf/d02/pred-stage3/01",
        "/D4/data/gvaillant/prep-uwrf/d02/pred-stage3/02"
    ]
    output_dir = '/D4/data/gvaillant/prep-uwrf/d02/pred-NYC-split/train'
    print(f"Output directory: {output_dir}")

    uWRF_combine_seq(input_dirs, output_dir)
    print("Done combining uWRF files!")

#Uncomment below to run:
#main()

In [12]:
def uWRF_val_test(input_dirs, output_dir):
    combined_datasets = []
    
    for input_dir in input_dirs:
        # Loop through each day's subdirectory within the monthly directory
        day_dirs = sorted(next(os.walk(input_dir))[1])  # Get list of day subdirectories
        for day_dir in day_dirs:
            day_path = os.path.join(input_dir, day_dir)  # Full path to each day's subdirectory
            nc_files = sorted(glob.glob(os.path.join(day_path, '*')))  # Gather all files in the daily subdirectory

            # Take the first half of the files
            half_length = len(nc_files) // 2
            #selected_files = nc_files[:half_length]  # Select first half of the sorted files (VALIDATION)
            selected_files = nc_files[half_length:] #Select second half of the sorted files (TESTING)
            
            # Open the selected files
            # Load each file into a dataset and add to the list
            datasets = [xr.open_dataset(file) for file in selected_files]
            combined_datasets.extend(datasets)

    # Concatenate all datasets along the 'time' dimension
    combined_dataset = xr.concat(combined_datasets, dim='time')

    # Generate output filename based on month range from input directories
    month_range = "-".join([os.path.basename(month_dir) for month_dir in input_dirs])
    output_file_name = f'uWRF_final_{month_range}.nc'
    output_file_path = os.path.join(output_dir, output_file_name)

    # Save the concatenated dataset to a NetCDF file
    combined_dataset.to_netcdf(output_file_path, encoding={'time': {'units': 'hours since 2019-10-11'}})
    print(f'Combined dataset saved to {output_file_path}')

def main():
    input_dirs = [
        "/D4/data/gvaillant/prep-uwrf/d02/pred-stage3/03"
    ]
    output_dir = '/D4/data/gvaillant/prep-uwrf/d02/pred-NYC-split/test' #change to val or test
    print(f"Output directory: {output_dir}")

    uWRF_combine_seq(input_dirs, output_dir)
    print("Done combining uWRF files!")

#Uncomment below to run:
#main()

# NAM functions

In [41]:
import netCDF4
import xarray as xr
import os
import glob
import numpy as np
from scipy.interpolate import griddata

def NAM_filter_and_match_dims(input_dir, output_dir, variables):
    # List all files in input directory
    file_list = [(os.path.join(input_dir, file), file) for file in os.listdir(input_dir) if file.endswith('.nc')]

    for file_path, file_name in file_list:  # Unpack the tuple
        try:
            # Step 1: Filter the variables using NAM_filter_vars logic
            with xr.open_dataset(file_path) as ds:
                existing_vars = {var: ds[var] for var in variables.keys() if var in ds}
                if not existing_vars:
                    print(f"No matching variables found in {file_name}.")
                    continue

                # Filter and rename variables
                ds_filtered = xr.Dataset(existing_vars).rename(variables)

                for var in ds_filtered.data_vars:
                    if 'time' in ds_filtered[var].dims:
                        dims = ('time',) + tuple(d for d in ds_filtered[var].dims if d != 'time')
                        ds_filtered[var] = ds_filtered[var].transpose(*dims)

                for orig_var, new_var in variables.items():
                    if orig_var in ds:
                        ds_filtered[new_var].attrs = ds[orig_var].attrs

                ds_filtered.attrs = ds.attrs

                # Change longitude values to be in degrees west
                if 'longitude' in ds_filtered:
                    lon = ds_filtered['longitude'].values
                    lon = np.where(lon > 180, lon - 360, lon)
                    ds_filtered['longitude'].values = lon
                    ds_filtered['longitude'].attrs['units'] = 'degrees_west'
        
        except Exception as e:
            print(f"Error processing file {file_name}: {e}")
            continue
        
        # Step 2: Interpolate variables using NAM_match_dims logic
        latitudes = ds_filtered['latitude'].values  # Shape: (67, 71)
        longitudes = ds_filtered['longitude'].values  # Shape: (67, 71)
        time = ds_filtered['time']
        
        # Save all the attributes for each variable
        lat_attrs = ds_filtered['latitude'].attrs
        lon_attrs = ds_filtered['longitude'].attrs
        time_attrs = ds_filtered['time'].attrs
        
        # Flatten latitude and longitude for interpolation
        points = np.array([(lon, lat) for lat_row, lon_row in zip(latitudes, longitudes) for lat, lon in zip(lat_row, lon_row)])
        
        # Define the new latitude and longitude grid
        new_latitudes = np.linspace(np.min(latitudes), np.max(latitudes), num=67)
        new_longitudes = np.linspace(np.min(longitudes), np.max(longitudes), num=67)
        
        # Create new meshgrid
        new_lon_grid, new_lat_grid = np.meshgrid(new_longitudes, new_latitudes)
        
        new_vars = {}
        
        for var_name in ds_filtered.data_vars:
            var = ds_filtered[var_name]
            new_var_list = []
            
            for t in range(len(var.time)):
                weather_variable = var.values[t, :, :]  # Shape (67, 71)
                
                # Flatten the weather variable data
                values = weather_variable.flatten()
                
                # Interpolate the data onto the new grid
                new_weather_variable = griddata(points, values, (new_lon_grid, new_lat_grid), method='linear')
                
                # Append the interpolated data for the current time step
                new_var_list.append(new_weather_variable)
            
            # Stack the new variables along the time dimension
            new_vars[var_name] = (['time', 'latitude', 'longitude'], np.stack(new_var_list))
        
        # Create a new xarray Dataset
        new_ds = xr.Dataset(
            new_vars, coords={'latitude': new_latitudes,
                              'longitude': new_longitudes,
                              'time': time.values})
        
        # Add the original variable attributes
        new_ds['time'].attrs.update(time_attrs)
        new_ds['latitude'].attrs.update(lat_attrs)
        new_ds['longitude'].attrs.update(lon_attrs)

        # Add global attributes
        new_ds.attrs.update(ds_filtered.attrs)

        #Saving files
        filename = os.path.basename(file_name)
        output_file = os.path.join(output_dir, filename)
        print(f"Saving file to: {output_file}")  # Print the output file path
        new_ds.to_netcdf(output_file)

def main():
    input_dir = "/D4/data/gvaillant/NAM-2019-netcdf/03"
    output_dir = "/D4/data/gvaillant/NAM/2019/match-pred/03"
    variables = {'TMP_2maboveground': 'T2', 'PRES_surface': 'PRES'}
    print(f"Output directory: {output_dir}")
    
    NAM_filter_and_match_dims(input_dir, output_dir, variables)
    
    print("Done processing stage1 NAM files!")

#main()

In [21]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os
#need to get it cut down to around the same area as uWRF

def NAM_spatial_filter(input_dir, output_dir):
    """
    Function to load the NAM dataset and spatially cut it to the uWRF bounds.
    
    Args:
    - input_dir (str): Path to the NAM directory
    - output_dir (str): Path to new directory to store filtered NAM data
    Returns:
    - None: Saves to the output directory.
    """
    #Bounds to match up with uWRF (but a little higher)
    #min_lat = 39.7
    #max_lat = 43.2
    #min_lon = -76.9
    #max_lon = -72.5

    #Bounds to match up with uWRF covering only NYC
    min_lat = 40.380194091796874
    max_lat = 41.090039825439455
    min_lon = -74.34615478515624
    max_lon = -73.495703125

    for file_name in os.listdir(input_dir):
        print(f"Processing file: {file_name}")
        if file_name.endswith('.nc'):

            file_path = os.path.join(input_dir, file_name)
            dataset = xr.open_dataset(file_path)
            
            #Extract latitude and longitude variables
            lat_var = 'latitude'
            lon_var = 'longitude'
            lat = dataset[lat_var].values
            lon = dataset[lon_var].values
            
            #Filter the data based off of the spatial bounds
            filtered_data = dataset.where(
                (dataset[lat_var] >= min_lat) & (dataset[lat_var] <= max_lat) &
                (dataset[lon_var] >= min_lon) & (dataset[lon_var] <= max_lon), drop=True)

            output_file_path = os.path.join(output_dir, file_name)
            filtered_data.to_netcdf(output_file_path)

def main():
    input_dir = "/D4/data/gvaillant/NAM/2019/match-pred/03"
    output_dir = "/D4/data/gvaillant/NAM/2019/match-NYC-cut/03"
    print(f"Output directory: {output_dir}")
    
    NAM_spatial_filter(input_dir, output_dir)
    
    print("Done spatially filtering the NAM files!")

#main()

In [18]:
import os
import xarray as xr

def NAM_combine_seq(input_dirs, output_dir):
    # Given two directories, combine files sequentially
    combined_datasets = []
    
    for input_dir in input_dirs:
        nc_files = sorted([os.path.join(input_dir, file) for file in os.listdir(input_dir) if file.endswith('.nc')])
        datasets = [xr.open_dataset(nc_file) for nc_file in nc_files]
        combined_datasets.extend(datasets)  # Add datasets sequentially

    # Concatenate along the time dimension
    combined_dataset = xr.concat(combined_datasets, dim='time')
    
    # Generate the output filename based on the directories' month identifiers
    month_range = "-".join([os.path.basename(dir) for dir in input_dirs])
    output_file_name = f'NAM_final_{month_range}.nc'
    output_file_path = os.path.join(output_dir, output_file_name)
    
    # Save the combined dataset
    combined_dataset.to_netcdf(output_file_path, encoding={'time': {'units': 'hours since 2019-1-1'}})
    print(f"Saved combined dataset to: {output_file_path}")

def main():

    input_dirs = [
        "/D4/data/gvaillant/NAM/2019/match-NYC-cut/01",
        "/D4/data/gvaillant/NAM/2019/match-NYC-cut/02"
    ]
    
    output_dir = "/D4/data/gvaillant/NAM/2019/match-NYC-final/train"
    print(f"Output directory: {output_dir}")
    
    NAM_combine_seq(input_dirs, output_dir)
    
    print("Done combining the first two months of NAM files!")

#main()

Output directory: /D4/data/gvaillant/NAM/2019/match-NYC-final/train
Saved combined dataset to: /D4/data/gvaillant/NAM/2019/match-NYC-final/train/NAM_final_01-02.nc
Done combining the first two months of NAM files!


In [20]:
import os
import xarray as xr

def NAM_val_test(input_dirs, output_dir):
    # Given two directories, combine files sequentially
    combined_datasets = []
    
    for input_dir in input_dirs:
        # Get sorted list of .nc files
        nc_files = sorted([os.path.join(input_dir, file) for file in os.listdir(input_dir) if file.endswith('.nc')])
        
        # Take the first half of the files
        half_length = len(nc_files) // 2
        #selected_files = nc_files[:half_length]  # Select first half of the sorted files (val)
        selected_files = nc_files[half_length:] #Select second half of the sorted files (test)
        # Open the selected files
        datasets = [xr.open_dataset(nc_file) for nc_file in selected_files]
        combined_datasets.extend(datasets)  # Add datasets sequentially

    # Concatenate along the time dimension
    combined_dataset = xr.concat(combined_datasets, dim='time')
    
    # Generate the output filename based on the directories' month identifiers
    month_range = "-".join([os.path.basename(dir) for dir in input_dirs])
    output_file_name = f'NAM_final_{month_range}.nc'
    output_file_path = os.path.join(output_dir, output_file_name)
    
    # Save the combined dataset
    combined_dataset.to_netcdf(output_file_path, encoding={'time': {'units': 'hours since 2019-1-1'}})
    print(f"Saved combined dataset to: {output_file_path}")

def main():
    input_dirs = [
        "/D4/data/gvaillant/NAM/2019/match-NYC-cut/03"
    ]
    
    output_dir = "/D4/data/gvaillant/NAM/2019/match-NYC-final/test"
    print(f"Output directory: {output_dir}")
    
    NAM_val_test(input_dirs, output_dir)
    
    print("Done combining the first half of the NAM files!")

#main()

Output directory: /D4/data/gvaillant/NAM/2019/match-NYC-final/test
Saved combined dataset to: /D4/data/gvaillant/NAM/2019/match-NYC-final/test/NAM_final_03.nc
Done combining the first half of the NAM files!


In [33]:
import xarray as xr
import numpy as np

uwrf_train_data = xr.open_dataset('/D4/data/gvaillant/prep-uwrf/d02/pred-split/train/uWRF_final_01-02.nc')
uwrf_val_data = xr.open_dataset('/D4/data/gvaillant/prep-uwrf/d02/pred-split/val/uWRF_final_03.nc')
uwrf_test_data = xr.open_dataset('/D4/data/gvaillant/prep-uwrf/d02/pred-split/test/uWRF_final_03.nc')
#--
nam_train_data = xr.open_dataset('/D4/data/gvaillant/NAM/2019/pred-final/train/NAM_final_01-02.nc')
nam_val_data = xr.open_dataset('/D4/data/gvaillant/NAM/2019/pred-final/val/NAM_final_03.nc')
nam_test_data = xr.open_dataset('/D4/data/gvaillant/NAM/2019/pred-final/test/NAM_final_03.nc')

nam_train_data = nam_train_data.rename({'PRES': 'PSFC'})
nam_val_data = nam_val_data.rename({'PRES': 'PSFC'})
nam_test_data = nam_test_data.rename({'PRES': 'PSFC'})

# Function to align datasets
def align_datasets(uwrf_data, nam_data):
    # uWRF grid dimensions
    uwrf_shape = uwrf_data.T2.shape  # Assuming T2 is representative of the shape
    uwrf_lons = uwrf_data.longitude
    uwrf_lats = uwrf_data.latitude

    # Assign number of uWRF cells per NAM cell
    uwrf_cells_per_lon = 4 #when using d03 we can use 12 bc we go from 12km to 1km
    uwrf_cells_per_lat = 4

    # Calculate new NAM grid dimensions
    new_nam_lon_count = uwrf_shape[2] // uwrf_cells_per_lon
    new_nam_lat_count = uwrf_shape[1] // uwrf_cells_per_lat

    # Function to aggregate 4x4 uWRF cells into one NAM cell
    def aggregate_4x4_grid(data):
        reshaped = data.reshape(
            data.shape[0],  # Time dimension remains unchanged
            new_nam_lat_count, uwrf_cells_per_lat, 
            new_nam_lon_count, uwrf_cells_per_lon
        )
        aggregated = reshaped.mean(axis=(2, 4))  # Aggregate over latitude and longitude cells
        return aggregated

    # Determine the minimum time dimension between NAM and uWRF
    min_time_steps = min(nam_data.time.size, uwrf_data.time.size)

    # Slice both datasets to the same time dimension
    nam_data_sliced = nam_data.isel(time=slice(0, min_time_steps))
    uwrf_data_sliced = uwrf_data.isel(time=slice(0, min_time_steps))

    # Initialize aligned data
    aligned_data = {}

    # Process both T2 and PRES
    for var_name in ['T2', 'PSFC']:
        if var_name in uwrf_data_sliced and var_name in nam_data_sliced:
            uwrf_var = uwrf_data_sliced[var_name].values
            aggregated_var = aggregate_4x4_grid(uwrf_var)
            aligned_data[var_name] = (['time', 'latitude', 'longitude'], aggregated_var)
        else:
            raise ValueError(f"Variable '{var_name}' not found in one of the datasets.")

    # Create a new dataset with aligned data
    aligned_nam = xr.Dataset(
        data_vars=aligned_data,
        coords={
            'time': nam_data_sliced.time,
            'latitude': uwrf_lats[::uwrf_cells_per_lat][:new_nam_lat_count],
            'longitude': uwrf_lons[::uwrf_cells_per_lon][:new_nam_lon_count]
        },
        attrs=nam_data.attrs
    )

    return aligned_nam

#Uncomment the lines below to run the code:
#aligned_nam_data = align_datasets(uwrf_test_data, nam_test_data)
#aligned_nam_data.to_netcdf("/home/gvaillant1/aligned-data/aligned_nam_val_data.nc")
#print("Done")

Done


In [24]:
## REGRIDDING NYC AREA:

import xarray as xr
import numpy as np

#this the one i used
uwrf_train_data = xr.open_dataset('/D4/data/gvaillant/prep-uwrf/d02/pred-NYC-split/train/uWRF_final_01-02.nc')
uwrf_val_data = xr.open_dataset('/D4/data/gvaillant/prep-uwrf/d02/pred-NYC-split/val/uWRF_final_03.nc')
uwrf_test_data = xr.open_dataset('/D4/data/gvaillant/prep-uwrf/d02/pred-NYC-split/test/uWRF_final_03.nc')
#--
nam_train_data = xr.open_dataset('/D4/data/gvaillant/NAM/2019/match-NYC-final/train/NAM_final_01-02.nc')
nam_val_data = xr.open_dataset('/D4/data/gvaillant/NAM/2019/match-NYC-final/val/NAM_final_03.nc')
nam_test_data = xr.open_dataset('/D4/data/gvaillant/NAM/2019/match-NYC-final/test/NAM_final_03.nc')

nam_train_data = nam_train_data.rename({'PRES': 'PSFC'})
nam_val_data = nam_val_data.rename({'PRES': 'PSFC'})
nam_test_data = nam_test_data.rename({'PRES': 'PSFC'})

# Function to align datasets
def align_datasets(uwrf_data, nam_data):
    # uWRF grid dimensions
    uwrf_shape = uwrf_data.T2.shape  # Assuming T2 is representative of the shape
    uwrf_lons = uwrf_data.longitude
    uwrf_lats = uwrf_data.latitude

    # Assign number of uWRF cells per NAM cell
    uwrf_cells_per_lon = 4 #when using d03 we can use 12 bc we go from 12km to 1km
    uwrf_cells_per_lat = 4

    # Calculate new NAM grid dimensions
    new_nam_lon_count = uwrf_shape[2] // uwrf_cells_per_lon
    new_nam_lat_count = uwrf_shape[1] // uwrf_cells_per_lat

    # Function to aggregate 4x4 uWRF cells into one NAM cell
    def aggregate_4x4_grid(data):
        reshaped = data.reshape(
            data.shape[0],  # Time dimension remains unchanged
            new_nam_lat_count, uwrf_cells_per_lat, 
            new_nam_lon_count, uwrf_cells_per_lon
        )
        aggregated = reshaped.mean(axis=(2, 4))  # Aggregate over latitude and longitude cells
        return aggregated

    # Determine the minimum time dimension between NAM and uWRF
    min_time_steps = min(nam_data.time.size, uwrf_data.time.size)

    # Slice both datasets to the same time dimension
    nam_data_sliced = nam_data.isel(time=slice(0, min_time_steps))
    uwrf_data_sliced = uwrf_data.isel(time=slice(0, min_time_steps))

    # Initialize aligned data
    aligned_data = {}

    # Process both T2 and PRES
    for var_name in ['T2', 'PSFC']:
        if var_name in uwrf_data_sliced and var_name in nam_data_sliced:
            uwrf_var = uwrf_data_sliced[var_name].values
            aggregated_var = aggregate_4x4_grid(uwrf_var)
            aligned_data[var_name] = (['time', 'latitude', 'longitude'], aggregated_var)
        else:
            raise ValueError(f"Variable '{var_name}' not found in one of the datasets.")

    # Create a new dataset with aligned data
    aligned_nam = xr.Dataset(
        data_vars=aligned_data,
        coords={
            'time': nam_data_sliced.time,
            'latitude': uwrf_lats[::uwrf_cells_per_lat][:new_nam_lat_count],
            'longitude': uwrf_lons[::uwrf_cells_per_lon][:new_nam_lon_count]
        },
        attrs=nam_data.attrs
    )

    return aligned_nam

#Uncomment the lines below to run the code:
#aligned_nam_data = align_datasets(uwrf_test_data, nam_test_data)
#aligned_nam_data.to_netcdf("/home/gvaillant1/aligned-NYC-data/aligned_nam_test_data.nc")
#print("Done")


Done
